In [1]:
# import pandas as pd
# %matplotlib inline
# import re
import numpy as np
import pandas as pd
# import string
# import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# import IPython.display as ipd
# import numpy
# import torch

In [5]:
data = pd.read_csv("train.csv.gz", header = None, nrows = 5)

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,88191,88192,88193,88194,88195,88196,88197,88198,88199,88200
0,-0.046627,-0.053661,-0.016606,-0.008460,-0.014638,-0.035218,-0.047178,-0.056229,-0.058350,-0.062226,...,-0.056585,-0.058592,-0.056084,-0.042918,-0.051984,-0.037560,-0.036154,-0.036979,-0.027567,5.0
1,0.000000,0.000712,0.001424,0.002135,0.002843,0.003550,0.004253,0.004954,0.005650,0.006341,...,-0.006341,-0.005650,-0.004954,-0.004253,-0.003550,-0.002843,-0.002135,-0.001424,-0.000712,0.0
2,0.015198,0.028053,0.027532,0.028745,0.024490,0.021514,0.017232,0.015572,0.014204,0.014561,...,-0.014135,-0.018649,-0.021481,-0.022629,-0.019144,-0.017117,-0.013521,-0.008887,-0.003531,8.0
3,-0.052935,-0.061702,-0.060131,-0.040481,0.029915,0.112537,0.198678,0.217514,0.180841,0.082558,...,0.013204,-0.003309,-0.013274,0.005676,0.031212,0.054739,0.070668,0.078661,0.097999,8.0
4,0.000218,0.006255,0.011420,0.015926,0.018090,0.018111,0.017685,0.017037,0.018973,0.020819,...,-0.007386,-0.009816,-0.012220,-0.013219,-0.001098,0.000035,0.000455,0.001339,0.003039,5.0


In [11]:
train = data.iloc[:,1:].values
print("Train shape",train.shape)
N_train = train.shape[0] #Same as N
NUM_SAMPLES = train.shape[1]-1 #Number of columns, sans the class label

Train shape (5, 88200)


In [ ]:
data = pd.read_csv("train_10.csv", header=None)

In [ ]:
data.head()

In [ ]:
# Just some re-shaping and dimension finding

N = 500 #Number of samples we are using
print("N:",N)
train = data.iloc[0:N,1:].values
print("Train shape",train.shape)
N_train = train.shape[0] #Same as N
NUM_SAMPLES = train.shape[1]-1 #Number of columns, sans the class label

X_train = train[:,:-1]
y_train = train[:,-1]
y_train = y_train.reshape(N_train,1)

In [ ]:
SAMPLE_RATE = 22050
song = X_train[1]
song_int = (32768*song).astype(np.int16)
print(song_int.dtype)
print(max(song_int))
print(min(song_int))
ipd.Audio(song_int, rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write


for i in range(0, X_train.shape[0]):
    write("WaveFiles/xtrain_"+str(i)+".wav", SAMPLE_RATE, (32768*X_train[i]).astype(np.int16))




In [ ]:
# JUST SOME FOURIER TRANSFORM PARAMETERS
SAMPLE_RATE = 22050
BINS_OCTAVE = 12*2
N_OCTAVES = 7
NUM_BINS = BINS_OCTAVE * N_OCTAVES

In [ ]:
# Given a wav time series, makes a mel spectrogram
# which is a short-time fourier transform with
# frequencies on the mel (log) scale.
def mel_spec(y):
    Q = librosa.cqt(y=y, sr=SAMPLE_RATE, bins_per_octave=BINS_OCTAVE,n_bins=NUM_BINS)
    Q_db = librosa.amplitude_to_db(Q,ref=np.max)
    return Q_db

In [ ]:
# This means that the spectrograms are 168 rows (frequencies)
# By 173 columns (time frames)
song = X_train[0]
test_spec = mel_spec(song)
print(test_spec.shape)
FEATS = test_spec.shape[0]
FRAMES = test_spec.shape[1]

In [ ]:
#This creates a tensor, in which we have N_train examples, each of which is a mel-frequency "image"

tmp_train = np.zeros((N_train,FEATS,FRAMES))
for i in range(N_train):
    tmp_train[i,:,:] = mel_spec(X_train[i])



In [ ]:
#Save the tensor, so that we can load it into other models
np.save('Data/xtrain_spec', tmp_train)
np.save('Data/ytrain_spec', y_train)

In [ ]:
librosa.display.specshow(tmp_train[0], y_axis='hz')